In [2]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

block_size = 8
batch_size = 4
max_iters = 10000
eval_iters = 2500
learning_rate = 3e-4
dropout = 0.2

In [3]:
with open('wizard_of_oz.txt','r',encoding= 'utf-8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)
print(len(chars))
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [4]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i ,ch in enumerate(chars)}



encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

encoded_hello = encode('hello')



decoded_hello = decode(encoded_hello)
print(decoded_hello)

data = torch.tensor(encode(text),dtype= torch.long)
print(data[:100])
print(len(data))

hello
tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])
232309


In [5]:
# n  = int(0.8 * len(data))
# # print(n)
# train_data = data[:n]
# # print('training data',train_data)
# val_data = data[n:]
# # print('val_data',val_data)



# x = train_data[:block_size]
# print('x data',x)
# y = train_data[1:block_size+1]
# print('y_data',y)

# for t in range(block_size):
#     context = x[:t+1]
#     target = y[t]
#     print('when input is ',context, 'target is ',target)



In [6]:
n = int(0.8 *len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size,(batch_size,))
    x = torch.stack([data[i :i + block_size] for i in ix])    
    y = torch.stack([data[i+1 :i + block_size + 1] for i in ix])
    x,y = x.to(device),y.to(device)
    return x, y

x,y = get_batch('train')
print('inputs: ')
print(x)
print('target:')
print(y)

inputs: 
tensor([[54, 71, 57,  1, 54, 72,  1, 61],
        [69, 58, 68, 69, 65, 58, 11,  3],
        [ 1, 72, 56, 71, 54, 73, 56, 61],
        [ 1, 54, 67, 57,  1, 73, 71, 62]], device='cuda:0')
target:
tensor([[71, 57,  1, 54, 72,  1, 61, 58],
        [58, 68, 69, 65, 58, 11,  3,  0],
        [72, 56, 71, 54, 73, 56, 61,  1],
        [54, 67, 57,  1, 73, 71, 62, 58]], device='cuda:0')


In [7]:
import torch.nn as nn
from torch.nn import functional as F

In [8]:

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [37]:
class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
        
            B,T,C = logits.shape
            # print(logits.shape,'logits shape')
            logits = logits.view(B*T,C)
            # print('yes this is logits ',logits)
            targets = targets.view(B*T)
            print(targets,'this is target')
            loss = F.cross_entropy(logits,targets)
            
        return logits,loss


    def generate(self, index, max_new_tokens):
        #index is (b,t) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:,-1,:] #becomes (B,C)
            #apply softmax to get probabilities
            probs = F.softmax(logits,dim= -1) # (B,1)
            # sample from the distribution 
            index_next = torch.multinomial(probs,num_samples=1)
            #append sampled index to the running sequence
            index = torch.cat((index,index_next),dim=1) # (B,T+1)

        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1),dtype = torch.long, device= device)
print(context,'index')
generated_chars = decode(m.generate(context,max_new_tokens=20)[0].tolist())
print(generated_chars)
            


tensor([[0]], device='cuda:0') index

﻿EIZ6I*IZH8fyK3YCEKT


In [38]:
## create a pytorch optimizer
optimizer = torch.optim.Adam(model.parameters(),lr= learning_rate)

for iter in range(1):
    if not iter == 1:
        print(iter)
        if iter % eval_iters == 0:
            losses = estimate_loss()
            # print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
        
        # sample a batch of data
        xb,yb = get_batch('train')
        # print(xb,yb)
        # evaluate the loss
        logits,loss = model.forward(xb,yb)
        optimizer.zero_grad(set_to_none = True)
        loss.backward()
        optimizer.step()

print(loss.item())

0


TypeError: 'int' object is not iterable

In [11]:
context = torch.zeros((1,1),dtype=torch.long,device = device)
generated_chars = decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)


btoryIDF_"H;
N-uzJ;NhM;Mi?'*O-*BbCgpD3NCnpDIFw'kFNR7!Oza eE?
I,[FEE?y1uRNcPjAA'qV'dO2*PySIVlVnrLnHX5H*o9a!GCuBydQByfWDis,WsJATO2xrH!WdyZ"H*AruAnxF!9S[IpDSmH6g5aWs,.Cj&;MvKXlx1u0xQT5]Mb7&)aUa,z
hdP[ffpcrPNDmaYMT[dk(T;GCm,-7M
]dPW:Td!9.t088xdYR-tkG
Jnf1[vPYrjD
TkB0:N 7!vP;﻿JTUrB]bN".jsGkj5oJJqf5 :'sR*[WQ4g[﻿﻿iYi;;5bIRht(&H﻿N [&aBqMLk 8'u_uKuN-SgZdru8qtzFwRDBokX2shlX,U]W67989﻿lKHhyW6Ag[",-f﻿:JV3Q'L6??rLoEXx30&[2S6hjxqiCRXOn*AAJBGCmaSJi!fwHfwk[﻿J;[tVfIzynT7!W9R7!E!h1BH. Hl(hlQy;;NbG*??yd:B]m5WIfnaW_
